<p style="text-align:center">
    <a href="https://skills.network/?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDS0321ENSkillsNetwork26802033-2022-01-01" target="_blank">
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/assets/logos/SN_web_lightmode.png" width="200" alt="Skills Network Logo">
    </a>
</p>


# **Hands-on Lab: Interactive Visual Analytics with Folium**


Estimated time needed: **40** minutes


The launch success rate may depend on many factors such as payload mass, orbit type, and so on. It may also depend on the location and proximities of a launch site, i.e., the initial position of rocket trajectories. Finding an optimal location for building a launch site certainly involves many factors and hopefully we could discover some of the factors by analyzing the existing launch site locations.


In the previous exploratory data analysis labs, you have visualized the SpaceX launch dataset using `matplotlib` and `seaborn` and discovered some preliminary correlations between the launch site and success rates. In this lab, you will be performing more interactive visual analytics using `Folium`.


## Objectives


This lab contains the following tasks:

*   **TASK 1:** Mark all launch sites on a map
*   **TASK 2:** Mark the success/failed launches for each site on the map
*   **TASK 3:** Calculate the distances between a launch site to its proximities

After completed the above tasks, you should be able to find some geographical patterns about launch sites.


Let's first import required Python packages for this lab:


In [4]:
# Option 1: If you're in a standard Jupyter environment (not JupyterLite)
# Use pip instead of piplite
import sys
import subprocess

# Install packages using pip
subprocess.check_call([sys.executable, '-m', 'pip', 'install', 'folium'])
subprocess.check_call([sys.executable, '-m', 'pip', 'install', 'pandas'])

# Option 2: If you need to use conda
# !conda install -c conda-forge folium -y
# !conda install -c conda-forge pandas -y

# After installation, import the packages
import folium
import pandas as pd

In [5]:
import folium
import pandas as pd

In [6]:
# Import folium MarkerCluster plugin
from folium.plugins import MarkerCluster
# Import folium MousePosition plugin
from folium.plugins import MousePosition
# Import folium DivIcon plugin
from folium.features import DivIcon

If you need to refresh your memory about folium, you may download and refer to this previous folium lab:


[Generating Maps with Python](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DV0101EN-SkillsNetwork/labs/v4/DV0101EN-Exercise-Generating-Maps-in-Python.ipynb)


In [ ]:
## Task 1: Mark all launch sites on a map


First, let's try to add each site's location on a map using site's latitude and longitude coordinates


The following dataset with the name `spacex_launch_geo.csv` is an augmented dataset with latitude and longitude added for each site.


In [8]:
# Download and read the `spacex_launch_geo.csv`
import pandas as pd
import requests
import io

URL = 'https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_geo.csv'

# Use requests instead of js.fetch
response = requests.get(URL)
spacex_csv_file = io.BytesIO(response.content)
spacex_df = pd.read_csv(spacex_csv_file)

# Now you can work with spacex_df

Now, you can take a look at what are the coordinates for each site.


In [9]:
# Select relevant sub-columns: `Launch Site`, `Lat(Latitude)`, `Long(Longitude)`, `class`
spacex_df = spacex_df[['Launch Site', 'Lat', 'Long', 'class']]
launch_sites_df = spacex_df.groupby(['Launch Site'], as_index=False).first()
launch_sites_df = launch_sites_df[['Launch Site', 'Lat', 'Long']]
launch_sites_df

,Launch Site,Lat,Long
0,CCAFS LC-40,28.562302,-80.577356
1,CCAFS SLC-40,28.563197,-80.576820
2,KSC LC-39A,28.573255,-80.646895
3,VAFB SLC-4E,34.632834,-120.610745


Above coordinates are just plain numbers that can not give you any intuitive insights about where are those launch sites. If you are very good at geography, you can interpret those numbers directly in your mind. If not, that's fine too. Let's visualize those locations by pinning them on a map.


We first need to create a folium `Map` object, with an initial center location to be NASA Johnson Space Center at Houston, Texas.


In [10]:
# Start location is NASA Johnson Space Center
nasa_coordinate = [29.559684888503615, -95.0830971930759]
site_map = folium.Map(location=nasa_coordinate, zoom_start=10)

We could use `folium.Circle` to add a highlighted circle area with a text label on a specific coordinate. For example,


In [11]:
# Create a blue circle at NASA Johnson Space Center's coordinate with a popup label showing its name
circle = folium.Circle(nasa_coordinate, radius=1000, color='#d35400', fill=True).add_child(folium.Popup('NASA Johnson Space Center'))
# Create a blue circle at NASA Johnson Space Center's coordinate with a icon showing its name
marker = folium.map.Marker(
    nasa_coordinate,
    # Create an icon as a text label
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % 'NASA JSC',
        )
    )
site_map.add_child(circle)
site_map.add_child(marker)

and you should find a small yellow circle near the city of Houston and you can zoom-in to see a larger circle.


Now, let's add a circle for each launch site in data frame `launch_sites`


*TODO:*  Create and add `folium.Circle` and `folium.Marker` for each launch site on the site map


An example of folium.Circle:


`folium.Circle(coordinate, radius=1000, color='#000000', fill=True).add_child(folium.Popup(...))`


An example of folium.Marker:


`folium.map.Marker(coordinate, icon=DivIcon(icon_size=(20,20),icon_anchor=(0,0), html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % 'label', ))`


In [14]:
import pandas as pd
import folium
from folium.features import DivIcon

# ---------- 1) Ensure `launch_sites` exists ----------
if 'launch_sites' not in globals():
    built = False
    # Try to build from any DataFrame you already have (e.g., spacex_df)
    dfs_in_memory = [v for v in globals().values() if isinstance(v, pd.DataFrame)]
    for df0 in dfs_in_memory:
        # Try to find likely columns
        site_col = next((c for c in df0.columns if 'launch' in c.lower() and 'site' in c.lower()), None)
        lat_col  = next((c for c in df0.columns if c.lower() in ['lat','latitude']), None)
        lon_col  = next((c for c in df0.columns if c.lower() in ['lon','long','longitude']), None)
        if site_col and lat_col and lon_col:
            tmp = (df0[[site_col, lat_col, lon_col]]
                   .dropna()
                   .drop_duplicates()
                   .rename(columns={site_col: 'Launch Site', lat_col: 'Lat', lon_col: 'Long'}))
            # If multiple rows per site, take mean coords
            launch_sites = (tmp.groupby('Launch Site', as_index=False)
                              .agg({'Lat': 'mean', 'Long': 'mean'}))
            built = True
            break

    if not built:
        # Fallback: known coordinates used in the lab
        launch_sites = pd.DataFrame([
            {'Launch Site': 'CCAFS SLC-40 (Cape Canaveral)', 'Lat': 28.5618571, 'Long': -80.577366},
            {'Launch Site': 'KSC LC-39A (Kennedy)',          'Lat': 28.5732550, 'Long': -80.646895},
            {'Launch Site': 'VAFB SLC-4E (Vandenberg)',      'Lat': 34.6320930, 'Long': -120.610829},
        ])

# ---------- 2) Add Circle + Marker for each site to your existing `site_map` ----------
# If you don't have site_map yet, uncomment the next 3 lines:
# center_lat = launch_sites['Lat'].mean()
# center_lon = launch_sites['Long'].mean()
# site_map = folium.Map(location=[center_lat, center_lon], zoom_start=4, tiles="CartoDB positron")

for _, r in launch_sites.iterrows():
    coord = [float(r['Lat']), float(r['Long'])]

    # Circle with popup
    folium.Circle(
        location=coord,
        radius=1000,     # meters
        color='#000000',
        weight=2,
        fill=True,
        fill_opacity=0.25
    ).add_child(
        folium.Popup(f"<b>{r['Launch Site']}</b><br>Lat: {r['Lat']:.6f}, Lon: {r['Long']:.6f}", max_width=250)
    ).add_to(site_map)

    # Label marker
    folium.Marker(
        location=coord,
        icon=DivIcon(
            icon_size=(150, 24),
            icon_anchor=(0, 0),
            html=f'<div style="font-size:12px; color:#d35400;"><b>{r["Launch Site"]}</b></div>'
        ),
        tooltip=r['Launch Site']
    ).add_to(site_map)

# Fit bounds so all markers are visible
site_map.fit_bounds([
    [launch_sites['Lat'].min(),  launch_sites['Long'].min()],
    [launch_sites['Lat'].max(),  launch_sites['Long'].max()]
])

site_map


The generated map with marked launch sites should look similar to the following:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_markers.png">
</center>


Now, you can explore the map by zoom-in/out the marked areas
, and try to answer the following questions:

*   Are all launch sites in proximity to the Equator line?
*   Are all launch sites in very close proximity to the coast?

Also please try to explain your findings.


In [ ]:
# Task 2: Mark the success/failed launches for each site on the map


Next, let's try to enhance the map by adding the launch outcomes for each site, and see which sites have high success rates.
Recall that data frame spacex_df has detailed launch records, and the `class` column indicates if this launch was successful or not


In [15]:
spacex_df.tail(10)

,Launch Site,Lat,Long,class
46,KSC LC-39A,28.573255,-80.646895,1
47,KSC LC-39A,28.573255,-80.646895,1
48,KSC LC-39A,28.573255,-80.646895,1
49,CCAFS SLC-40,28.563197,-80.576820,1
50,CCAFS SLC-40,28.563197,-80.576820,1
51,CCAFS SLC-40,28.563197,-80.576820,0
52,CCAFS SLC-40,28.563197,-80.576820,0
53,CCAFS SLC-40,28.563197,-80.576820,0
54,CCAFS SLC-40,28.563197,-80.576820,1
55,CCAFS SLC-40,28.563197,-80.576820,0


Next, let's create markers for all launch records.
If a launch was successful `(class=1)`, then we use a green marker and if a launch was failed, we use a red marker `(class=0)`


Note that a launch only happens in one of the four launch sites, which means many launch records will have the exact same coordinate. Marker clusters can be a good way to simplify a map containing many markers having the same coordinate.


Let's first create a `MarkerCluster` object


In [16]:
marker_cluster = MarkerCluster()


*TODO:* Create a new column in `spacex_df` dataframe called `marker_color` to store the marker colors based on the `class` value


In [18]:

import folium
from folium.plugins import MarkerCluster

# If you haven't: create the map centered on NASA coordinate
# site_map = folium.Map(location=nasa_coordinate, zoom_start=5, tiles="CartoDB positron")

# Safety: make sure marker_color exists
if 'marker_color' not in spacex_df.columns:
    spacex_df['marker_color'] = spacex_df['class'].map({1: 'green', 0: 'red'})

# Try common column names
lat_col = next((c for c in ['Lat','Latitude','lat','latitude'] if c in spacex_df.columns), None)
lon_col = next((c for c in ['Long','Lon','Longitude','long','lon','longitude'] if c in spacex_df.columns), None)
site_col = next((c for c in ['Launch Site','Launch_Site','site','Site'] if c in spacex_df.columns), None)

if not all([lat_col, lon_col, site_col]):
    raise ValueError("Columns for latitude/longitude/site not found in spacex_df.")

# Optional: cluster markers so the map stays tidy
cluster = MarkerCluster(name="Launch Records").add_to(site_map)

# Add one marker per launch record with success/failure color
for _, r in spacex_df.dropna(subset=[lat_col, lon_col]).iterrows():
    success = (r['class'] == 1)
    icon_name = 'ok-sign' if success else 'remove-sign'
    tooltip = f"{r[site_col]} — {'Success' if success else 'Failure'}"

    # Nice popup with a few fields if present
    payload = r.get('Payload Mass (kg)', r.get('Payload_Mass_kg', 'N/A'))
    booster = r.get('Booster Version', r.get('Booster_Version', 'N/A'))
    date    = r.get('Date', 'N/A')
    popup_html = (
        f"<b>{r[site_col]}</b><br>"
        f"Date: {date}<br>"
        f"Outcome: {'Success' if success else 'Failure'}<br>"
        f"Payload: {payload} kg<br>"
        f"Booster: {booster}"
    )

    folium.Marker(
    location=[float(r[lat_col]), float(r[lon_col])],
    tooltip=tooltip,
    popup=folium.Popup(popup_html, max_width=280),
    icon=folium.Icon(color=r['marker_color'], icon=icon_name)  # ← fixed
).add_to(cluster)






*TODO:* For each launch result in `spacex_df` data frame, add a `folium.Marker` to `marker_cluster`


In [19]:
import folium
from folium.plugins import MarkerCluster

# 1) Create & attach the cluster ONCE (remove duplicate add_child lines)
marker_cluster = MarkerCluster(name="Launch Records").add_to(site_map)

# 2) Ensure the success/failure color column exists
if 'marker_color' not in spacex_df.columns:
    spacex_df['marker_color'] = spacex_df['class'].map({1: 'green', 0: 'red'})

# 3) Column names (adjust if yours differ)
lat_col  = next((c for c in ['Lat','Latitude','lat','latitude'] if c in spacex_df.columns), None)
lon_col  = next((c for c in ['Long','Lon','Longitude','long','lon','longitude'] if c in spacex_df.columns), None)
site_col = next((c for c in ['Launch Site','Launch_Site','Site','site'] if c in spacex_df.columns), None)

if not all([lat_col, lon_col, site_col]):
    raise ValueError("Could not find lat/lon/site columns in spacex_df.")

# 4) Add one marker per record with success/failure icon color
for index, record in spacex_df.dropna(subset=[lat_col, lon_col]).iterrows():
    icon_name = 'ok-sign' if record['class'] == 1 else 'remove-sign'  # Glyphicons
    tooltip = f"{record[site_col]} — {'Success' if record['class']==1 else 'Failure'}"

    # (Optional) richer popup if fields exist
    payload = record.get('Payload Mass (kg)', record.get('Payload_Mass_kg', 'N/A'))
    booster = record.get('Booster Version', record.get('Booster_Version', 'N/A'))
    date    = record.get('Date', 'N/A')
    popup_html = (
        f"<b>{record[site_col]}</b><br>"
        f"Date: {date}<br>"
        f"Outcome: {'Success' if record['class']==1 else 'Failure'}<br>"
        f"Payload: {payload} kg<br>"
        f"Booster: {booster}"
    )

    marker = folium.Marker(
        location=[float(record[lat_col]), float(record[lon_col])],
        tooltip=tooltip,
        popup=folium.Popup(popup_html, max_width=280),
        icon=folium.Icon(color='white', icon_color=record['marker_color'], icon=icon_name)
    )
    marker_cluster.add_child(marker)

# (Optional) Fit map to all points
site_map.fit_bounds([
    [spacex_df[lat_col].min(), spacex_df[lon_col].min()],
    [spacex_df[lat_col].max(), spacex_df[lon_col].max()]
])

site_map


Your updated map may look like the following screenshots:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_marker_cluster.png">
</center>


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_marker_cluster_zoomed.png">
</center>


From the color-labeled markers in marker clusters, you should be able to easily identify which launch sites have relatively high success rates.


In [ ]:
# TASK 3: Calculate the distances between a launch site to its proximities


Next, we need to explore and analyze the proximities of launch sites.


Let's first add a `MousePosition` on the map to get coordinate for a mouse over a point on the map. As such, while you are exploring the map, you can easily find the coordinates of any points of interests (such as railway)


In [20]:
from folium.plugins import MousePosition

# Add mouse position readout (Lat, Long) to the map
formatter = "function(num) {return L.Util.formatNum(num, 5);};"

MousePosition(
    position='topright',
    separator='  Long: ',
    empty_string='NaN',
    lng_first=False,            # set True to show Long before Lat
    num_digits=20,              # max precision (formatter controls display to 5)
    prefix='Lat: ',
    lat_formatter=formatter,
    lng_formatter=formatter,
).add_to(site_map)

site_map


Now zoom in to a launch site and explore its proximity to see if you can easily find any railway, highway, coastline, etc. Move your mouse to these points and mark down their coordinates (shown on the top-left) in order to the distance to the launch site.


Now zoom in to a launch site and explore its proximity to see if you can easily find any railway, highway, coastline, etc. Move your mouse to these points and mark down their coordinates (shown on the top-left) in order to the distance to the launch site.


In [22]:
# 1) Define the distance function (Haversine, returns km)
from math import sin, cos, sqrt, atan2, radians

def calculate_distance(lat1, lon1, lat2, lon2, R=6371.0):
    lat1, lon1, lat2, lon2 = map(radians, [lat1, lon1, lat2, lon2])
    dlon, dlat = lon2 - lon1, lat2 - lat1
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    return 2 * R * atan2(sqrt(a), sqrt(1 - a))

# 2) Column names (adjust if yours differ)
lat_col  = next(c for c in ['Lat','Latitude','lat','latitude'] if c in launch_sites.columns)
lon_col  = next(c for c in ['Long','Lon','Longitude','long','lon','longitude'] if c in launch_sites.columns)
name_col = next(c for c in ['Launch Site','Launch_Site','Site','site'] if c in launch_sites.columns)

# 3) Distance from nasa_coordinate to each site
nasa_lat, nasa_lon = nasa_coordinate
launch_sites['dist_km'] = launch_sites.apply(
    lambda r: calculate_distance(nasa_lat, nasa_lon, float(r[lat_col]), float(r[lon_col])),
    axis=1
)

# 4) Nearest site + show table
nearest = launch_sites.loc[launch_sites['dist_km'].idxmin()]
print(f"Nearest site: {nearest[name_col]} — {nearest['dist_km']:.2f} km")
display(launch_sites[[name_col, lat_col, lon_col, 'dist_km']].sort_values('dist_km').reset_index(drop=True))

# 5) (Optional) draw a line on your Folium map
import folium
folium.PolyLine(
    locations=[nasa_coordinate, [nearest[lat_col], nearest[lon_col]]],
    tooltip=f"{nearest[name_col]} — {nearest['dist_km']:.1f} km",
    weight=3
).add_to(site_map)

site_map


Nearest site: KSC LC-39A — 1406.43 km


,Launch Site,Lat,Long,dist_km
0,KSC LC-39A,28.573255,-80.646895,1406.434271
1,CCAFS LC-40,28.562302,-80.577356,1413.328592
2,CCAFS SLC-40,28.563197,-80.576820,1413.366602
3,VAFB SLC-4E,34.632834,-120.610745,2462.753721


*TODO:* Mark down a point on the closest coastline using MousePosition and calculate the distance between the coastline point and the launch site.


In [23]:
# --- 1) Pick a launch site (from your launch_sites DataFrame) ---
site_name = "KSC LC-39A"   # <-- change to a substring that matches your site
row = (launch_sites[launch_sites['Launch Site'].str.contains(site_name, case=False, na=False)]
       .iloc[0])
launch_site_lat, launch_site_lon = float(row['Lat']), float(row['Long'])

# --- 2) Paste the coastline coordinates you read from the map hover (MousePosition) ---
coastline_lat, coastline_lon = 28.56367, -80.57163   # <-- replace with your picked coastline point

# --- 3) Distance function (Haversine, km) ---
from math import sin, cos, sqrt, atan2, radians
def calculate_distance(lat1, lon1, lat2, lon2, R=6371.0):
    lat1, lon1, lat2, lon2 = map(radians, [lat1, lon1, lat2, lon2])
    dlon, dlat = lon2 - lon1, lat2 - lat1
    a = sin(dlat/2)**2 + cos(lat1)*cos(lat2)*sin(dlon/2)**2
    return 2*R*atan2(sqrt(a), sqrt(1 - a))

# --- 4) Compute distance ---
distance_coastline = calculate_distance(launch_site_lat, launch_site_lon,
                                        coastline_lat, coastline_lon)
print(f"{row['Launch Site']}: distance to coastline = {distance_coastline:.2f} km")

# --- 5) Draw markers + line on the Folium map ---
import folium
from folium.features import DivIcon

# launch site marker
folium.Marker(
    [launch_site_lat, launch_site_lon],
    tooltip=row['Launch Site'],
    icon=folium.Icon(color="blue", icon="rocket", prefix="fa")
).add_to(site_map)

# coastline marker with label
folium.Marker(
    [coastline_lat, coastline_lon],
    tooltip="Coastline point",
    icon=DivIcon(icon_size=(160, 24), icon_anchor=(0, 0),
                 html='<div style="font-size:12px;color:#d35400;"><b>Coastline</b></div>')
).add_to(site_map)

# line between site and coastline
folium.PolyLine(
    [[launch_site_lat, launch_site_lon], [coastline_lat, coastline_lon]],
    tooltip=f"{distance_coastline:.2f} km",
    weight=3,
    dash_array='6,6'
).add_to(site_map)

# zoom to both points
site_map.fit_bounds([
    [min(launch_site_lat, coastline_lat), min(launch_site_lon, coastline_lon)],
    [max(launch_site_lat, coastline_lat), max(launch_site_lon, coastline_lon)],
])

site_map


KSC LC-39A: distance to coastline = 7.43 km


In [25]:
from folium.features import DivIcon
import folium

# Prereqs:
# - site_map exists (e.g., site_map = folium.Map(location=nasa_coordinate, zoom_start=6))
# - calculate_distance(lat1, lon1, lat2, lon2) is defined

launch_site_coord = [28.563197, -80.576820]   # CCAFS SLC-40
coastline_coord   = [28.56367,  -80.57163]    # closest coastline (from MousePosition)

# Compute distance (km)
dist_km = calculate_distance(launch_site_coord[0], launch_site_coord[1],
                             coastline_coord[0],   coastline_coord[1])

# Optional: show a line between site and coastline
folium.PolyLine([launch_site_coord, coastline_coord],
                tooltip=f"{dist_km:.2f} km", weight=3, dash_array="6,6").add_to(site_map)

# Launch site marker
folium.Marker(
    launch_site_coord,
    tooltip="Launch Site (CCAFS SLC-40)",
    icon=folium.Icon(color="green", icon="rocket", prefix="fa")
).add_to(site_map)

# Small dot at coastline point so the label has an anchor
folium.CircleMarker(coastline_coord, radius=4, color="#d35400", fill=True, fill_opacity=1).add_to(site_map)

# Distance label using DivIcon (offset so it doesn’t cover the dot)
folium.Marker(
    location=coastline_coord,
    icon=DivIcon(
        icon_size=(140, 20),
        icon_anchor=(10, -10),  # shift label a bit to avoid overlap
        html=f'<div style="font-size:12px;color:#d35400;"><b>{dist_km:.2f} km</b></div>',
    )
).add_to(site_map)

# Fit view to both points
site_map.fit_bounds([
    [min(launch_site_coord[0], coastline_coord[0]), min(launch_site_coord[1], coastline_coord[1])],
    [max(launch_site_coord[0], coastline_coord[0]), max(launch_site_coord[1], coastline_coord[1])]
])

site_map


*TODO:* Draw a `PolyLine` between a launch site to the selected coastline point


In [26]:
# assumes: site_map exists, calculate_distance(...) defined,
#          launch_site_coord = [lat, lon], coastline_coord = [lat, lon]

# 1) Distance (km)
dist_km = calculate_distance(launch_site_coord[0], launch_site_coord[1],
                             coastline_coord[0],   coastline_coord[1])

# 2) Draw the line with a tooltip
coordinates = [launch_site_coord, coastline_coord]
folium.PolyLine(
    locations=coordinates,
    weight=3, color='blue', dash_array='6,6',
    tooltip=f"Distance: {dist_km:.2f} km"
).add_to(site_map)

# 3) Optional: put a small text label at the midpoint
from folium.features import DivIcon
mid = [(launch_site_coord[0]+coastline_coord[0])/2,
       (launch_site_coord[1]+coastline_coord[1])/2]
folium.Marker(
    mid,
    icon=DivIcon(icon_size=(120,20), icon_anchor=(0,0),
                 html=f'<div style="font-size:12px;color:#1f77b4;"><b>{dist_km:.2f} km</b></div>')
).add_to(site_map)

# 4) Fit view to both points
site_map.fit_bounds([
    [min(launch_site_coord[0], coastline_coord[0]), min(launch_site_coord[1], coastline_coord[1])],
    [max(launch_site_coord[0], coastline_coord[0]), max(launch_site_coord[1], coastline_coord[1])]
])

site_map



Your updated map with distance line should look like the following screenshot:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_marker_distance.png">
</center>


*TODO:* Similarly, you can draw a line betwee a launch site to its closest city, railway, highway, etc. You need to use `MousePosition` to find the their coordinates on the map first


A railway map symbol may look like this:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/railway.png">
</center>


A highway map symbol may look like this:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/highway.png">
</center>


A city map symbol may look like this:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/city.png">
</center>


In [ ]:
# Create a marker with distance to a closest city, railway, highway, etc.
# Draw a line between the marker to the launch site


In [27]:
import folium
from folium.features import DivIcon

# --- Prereqs ---
# site_map already exists
# launch_site_coord = [lat, lon] of your launch site
# calculate_distance(lat1, lon1, lat2, lon2) is defined (Haversine, returns km)

# Example POIs (REPLACE with your coordinates)
pois = [
    {"name": "Closest City: Cape Canaveral", "coord": [28.3922, -80.6077], "icon": "building", "color": "blue"},
    {"name": "Closest Highway: FL-401",      "coord": [28.4151, -80.6466], "icon": "road",     "color": "darkpurple"},
    {"name": "Closest Railway",              "coord": [28.4035, -80.6312], "icon": "train",    "color": "gray"},
]

# Optional: group them
fg = folium.FeatureGroup(name="Nearest features").add_to(site_map)

# Ensure the launch site marker is visible
folium.Marker(
    launch_site_coord,
    tooltip="Launch Site",
    icon=folium.Icon(color="green", icon="rocket", prefix="fa")
).add_to(fg)

# Add each POI with distance label and a connecting line
for poi in pois:
    dist_km = calculate_distance(launch_site_coord[0], launch_site_coord[1],
                                 poi["coord"][0],       poi["coord"][1])

    # Line from site to POI (tooltip shows distance)
    folium.PolyLine(
        [launch_site_coord, poi["coord"]],
        color=poi["color"], weight=3, dash_array="6,6",
        tooltip=f"{poi['name']}: {dist_km:.2f} km"
    ).add_to(fg)

    # POI marker
    folium.Marker(
        poi["coord"],
        tooltip=poi["name"],
        popup=f"{poi['name']} — {dist_km:.2f} km",
        icon=folium.Icon(color=poi["color"], icon=poi["icon"], prefix="fa")
    ).add_to(fg)

    # Distance label near the POI (offset to avoid overlap)
    folium.Marker(
        poi["coord"],
        icon=DivIcon(
            icon_size=(140, 20), icon_anchor=(10, -10),
            html=f'<div style="font-size:12px;color:{poi["color"]};"><b>{dist_km:.2f} km</b></div>'
        )
    ).add_to(fg)

# Fit view to include site + all POIs
latitudes  = [launch_site_coord[0]] + [p["coord"][0] for p in pois]
longitudes = [launch_site_coord[1]] + [p["coord"][1] for p in pois]
site_map.fit_bounds([[min(latitudes), min(longitudes)], [max(latitudes), max(longitudes)]])

site_map


After you plot distance lines to the proximities, you can answer the following questions easily:

*   Are launch sites in close proximity to railways?
*   Are launch sites in close proximity to highways?
*   Are launch sites in close proximity to coastline?
*   Do launch sites keep certain distance away from cities?

Also please try to explain your findings.


# Next Steps:

Now you have discovered many interesting insights related to the launch sites' location using folium, in a very interactive way. Next, you will need to build a dashboard using Ploty Dash on detailed launch records.


## Authors


[Pratiksha Verma](https://www.linkedin.com/in/pratiksha-verma-6487561b1/)


<!--## Change Log--!>


<!--| Date (YYYY-MM-DD) | Version | Changed By      | Change Description      |
| ----------------- | ------- | -------------   | ----------------------- |
| 2022-11-09        | 1.0     | Pratiksha Verma | Converted initial version to Jupyterlite|--!>


### <h3 align="center"> IBM Corporation 2022. All rights reserved. <h3/>
